In [1]:
try:
    import os
    import sys
    import elasticsearch
    from elasticsearch import Elasticsearch, helpers
    import pandas as pd
    import json
    import os
    import sys
    import datetime
    from ast import literal_eval
    from tqdm import tqdm
    import numpy as np
    
    print('Imported all modules')
    
except Exception as e:
    print('Modules missing {}'.format(e))

Imported all modules


In [2]:
def connect_elasticsearch():
    es = None
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if es.ping():
        print('Yupiee  Connected ')
    else:
        print('Awww it could not connect!')
    return es
es = connect_elasticsearch()

# Create index

In [5]:
es.indices.create(index = 'my-food',ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my-food'}

In [6]:
res = es.indices.get_alias('*')
for name in res:
    print(name)

.kibana-event-log-7.16.0-000001
favorite_candy
.apm-agent-configuration
.kibana_task_manager_7.16.0_001
.kibana_7.16.0_001
my-food
.apm-custom-link
.tasks


C:\Users\shubh\anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: this request accesses system indices: [.apm-agent-configuration, .kibana_task_manager_7.16.0_001, .kibana_7.16.0_001, .apm-custom-link, .tasks], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


# Delete index

In [7]:
es.indices.delete(index = 'my-food',ignore=400)

{'acknowledged': True}

In [ ]:
#delete all indices
indices=es.indices.get_alias().keys()

for Name in indices:
    print("Deleted {} ".format(Name))
    es.indices.delete(index=Name, ignore=[400, 404])

In [8]:
res = es.indices.get_alias('*')
for name in res:
    print(name)

.kibana_task_manager_7.16.0_001
.kibana_7.16.0_001
.tasks
.apm-agent-configuration
.apm-custom-link
favorite_candy
.kibana-event-log-7.16.0-000001


C:\Users\shubh\anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: this request accesses system indices: [.kibana_task_manager_7.16.0_001, .kibana_7.16.0_001, .tasks, .apm-agent-configuration, .apm-custom-link], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


# Upload 2 sample json docs

In [10]:
e1 = {
    'first name': 'Shubham',
    'last name' : 'Singh',
    'about' : 'Data Scientist',
    'interest' : ['creating content','movies'],
}

e2 = {
    'first name': 'Shivam',
    'last name' : 'Singh',
    'about' : 'Cloud engineer',
    'interest' : ['Dance','music'],
}

In [11]:
es.indices.create(index = 'person',ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'person'}

In [12]:
#send doc to elastic serach
res1 = es.index(index='person', doc_type='people',body=e1,id=1)

C:\Users\shubh\anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
C:\Users\shubh\anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


In [13]:
res

{'.kibana_task_manager_7.16.0_001': {'aliases': {'.kibana_task_manager': {},
   '.kibana_task_manager_7.16.0': {}}},
 '.kibana_7.16.0_001': {'aliases': {'.kibana': {}, '.kibana_7.16.0': {}}},
 '.tasks': {'aliases': {}},
 '.apm-agent-configuration': {'aliases': {}},
 '.apm-custom-link': {'aliases': {}},
 'favorite_candy': {'aliases': {}},
 '.kibana-event-log-7.16.0-000001': {'aliases': {'.kibana-event-log-7.16.0': {'is_write_index': True,
    'is_hidden': True}}}}

In [14]:
res2 = es.index(index='person', doc_type='people',body=e2,id=2)

# Adding document

In [2]:
df=pd.read_csv('netflix_titles.csv')

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [5]:
#create an elastic search instance and connect it to the server
ENDPOINT = 'http://localhost:9200/'
es=Elasticsearch(timeout=600,hosts=ENDPOINT)

In [8]:
es.ping()

C:\Users\shubh\anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


True

In [9]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [10]:
df=df.dropna()

In [11]:
df.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [12]:
df.shape

(5332, 12)

In [13]:
#convert data into appropriate format
df2=df.to_dict('records')

In [14]:
df2[0]

{'show_id': 's8',
 'type': 'Movie',
 'title': 'Sankofa',
 'director': 'Haile Gerima',
 'cast': 'Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri',
 'country': 'United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia',
 'date_added': 'September 24, 2021',
 'release_year': 1993,
 'rating': 'TV-MA',
 'duration': '125 min',
 'listed_in': 'Dramas, Independent Movies, International Movies',
 'description': 'On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.'}

# convert data into ELK format

In [15]:
def generator(df2):
    for c, line in enumerate(df2):
        try:
            yield {
                '_index': 'elkdemo',
                '_type' :'_doc',
                '_id' :line.get('show_id', None),
                '_source' : {
                    'title' : line.get('title', ''),
                    'director' : line.get('director', ''),
                    'description' : line.get('description', ''),
                    'duration' : line.get('duration', None),
                    'cast':line.get('cast', None)

                }
            }
        except StopIteration:
            return
    

In [16]:
mycustom=generator(df2)

In [17]:
next(mycustom)

{'_index': 'elkdemo',
 '_type': '_doc',
 '_id': 's8',
 '_source': {'title': 'Sankofa',
  'director': 'Haile Gerima',
  'description': 'On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.',
  'duration': '125 min',
  'cast': 'Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri'}}

# Mappings(defining datatypes explicitly for the fields/columns) or Settings
By default, every string gets mapped twice as a text field and as a keyword multi-field. Each field type is primed for different types of requests.

- Text field type is designed for full-text searches.

- Keywordfield type is designed for exact searches, aggregations, and sorting.

SO use mapping to identify the exact type and use-case of each so that each string isn't indexed twice

Ever notice that when you search in Elasticsearch, it is not case sensitive or punctuation does not seem to matter? This is because text analysis occurs when your fields are indexed.

By default, strings are analyzed when it is indexed. The string is broken up into individual words also known as tokens. The analyzer further lowercases each token and removes punctuations.

- Remember, you CANNOT change the mapping of an existing field. If you do need to make changes to an existing field, you must create a new index with the desired mapping, then reindex all documents into the new index.

In [22]:
#if we don't define a datatype elastic search comes up with its own type
Settings ={
    'settings':{
        'number_of_shards':1,
        'number_of_replicas':0
    },
    'mappings':{
        'properties':{
            'director':{
                'type':'text'
            },
            'duration':{
                    'type':'text'
                }
            }
        }
    }

In [23]:
IndexName='elkdemo'
my=es.indices.create(index=IndexName, ignore=[400,404], body=Settings)
my

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'elkdemo'}

# Upload document and delete any existing indices

In [24]:
try:
    res = helpers.bulk(es,generator(df2))
    print('Working')
except Exception as e:
    pass

Working


# Running queries (below queries are to be run on http://localhost:5601/app/dev_tools#/console

In [25]:
'''GET elkdemo/_search - prints all documents and shown below is one of the records

      {
        "_index" : "elkdemo",
        "_type" : "_doc",
        "_id" : "s10",
        "_score" : 1.0,
        "_source" : {
          "title" : "The Starling",
          "director" : "Theodore Melfi",
          "description" : "A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward.",
          "duration" : "104 min",
          "cast" : "Melissa McCarthy, Chris O'Dowd, Kevin Kline, Timothy Olyphant, Daveed Diggs, Skyler Gisondo, Laura Harrier, Rosalind Chao, Kimberly Quinn, Loretta Devine, Ravi Kapoor"
        }
      }
      
      ### return just the title and cast
      GET elkdemo/_search
{
  "_source": ["title",'cast']
}

'''

'GET elkdemo/_search - prints all documents and shown below is one of the records\n\n      {\n        "_index" : "elkdemo",\n        "_type" : "_doc",\n        "_id" : "s10",\n        "_score" : 1.0,\n        "_source" : {\n          "title" : "The Starling",\n          "director" : "Theodore Melfi",\n          "description" : "A woman adjusting to life after a loss contends with a feisty bird that\'s taken over her garden — and a husband who\'s struggling to find a way forward.",\n          "duration" : "104 min",\n          "cast" : "Melissa McCarthy, Chris O\'Dowd, Kevin Kline, Timothy Olyphant, Daveed Diggs, Skyler Gisondo, Laura Harrier, Rosalind Chao, Kimberly Quinn, Loretta Devine, Ravi Kapoor"\n        }\n      }\n      \n      ### return just the title and cast\n      GET elkdemo/_search\n{\n  "_source": ["title",\'cast\']\n}\n\n'

In [26]:
GET elkdemo/_search
{
  "_source": ["title"]
}

SyntaxError: invalid syntax (<ipython-input-26-cba0ba0ce544>, line 1)

In [ ]:
# Find the exact match i.e all words and in same order
GET elkdemo/_search
{
  "_source": ["title","cast","director"],
  "size": 20,
  "query": {
    "match_phrase": {
      "title": "Norm of the North"
    }
  }
}

In [ ]:
# Perform tokenization and check is entire string (Norm of the North) or substrings (Norm, the North, North, of etc) exist

GET elkdemo/_search
{
  "_source": ["title","cast","director"],
  "size": 20,
  "query": {
    "match": {
      "title": "Norm of the North"
    }
  }
}

In [ ]:
# Aggregation - count the number of occurences of each cast

GET elkdemo/_search
{
  "size": 0,
  "aggs": {
    "cast_name": {
      "terms": {
        "field": "cast.keyword",
        "size": 10,
        "order": {
          "_count": "desc"
        }
      }
    }
  }
  
}

##Multiple aggregation

GET elkdemo/_search
{
  "size": 0,
  "aggs": {
    "description": { #name of the aggregation
      "terms": {
        "field": "description.keyword",
        "size": 10,
        "order": {
          "_count": "desc"
        }
      }
    },
    
      "cast_name": {
      "terms": {
        "field": "cast.keyword",
        "size": 10,
        "order": {
          "_count": "desc"
        }
      }
    }
  }
  
}

In [ ]:
# Boolean Search - paste the below cell data within any of the []
GET elkdemo/_search
{
  "size": 20,
  "query": {
    "bool": {
      "must": [], -> must contain both the terms i.e north and king
      "must_not": [], -> must not contain either of the terms mentioned i.e neither north nor king
      "filter": [], -> These place documents in either yes or no category. Ones that fall into the yes category are included in the hits.
      "should": [] -> does not have to match north or king. However, it if it does match, this document is given a higher score.
    }
  }
}

In [ ]:
{
        "match": {
          "title": {
            "query": "North"
          }
        }
      },
      {
        "match": {
          "title": {
            "query": "King"
          }
        }
      }

In [ ]:
# String query- i.e search for documents containing north or king

GET elkdemo/_search
{
  "query": {
    "query_string": {
      "query": "(north) OR (king)",
      "default_field": "title"
    }
  }
}

# 2 Ways to do Pagination or scrolling in Elastic Search

## Method 1

In [27]:
try:
    import os
    import sys
    
    import elasticsearch
    from elasticsearch import Elasticsearch 
    import pandas as pd
    
    print("All Modules Loaded ! ")
except Exception as e:
    print("Some Modules are Missing {}".format(e))
    

All Modules Loaded ! 


In [28]:
def connect_elasticsearch():
    es = None
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if es.ping():
        print('Yupiee  Connected ')
    else:
        print('Awww it could not connect!')
    return es
es = connect_elasticsearch()

Yupiee  Connected 


In [49]:
myquey = {
   "_source": [],
   "size": 10,
   "query": {
      "bool": {
         "must": [],
         "filter": [
            {
               "exists": {
                  "field": "director"
               }
            }
         ],
         "should": [
            {
               "match_phrase": {
                  "director": "Richard "
               }
            }
         ],
         "must_not": []
      }
   }
}

In [50]:
res = es.search(
  index = 'elkdemo', #name of the index name
  scroll = '2m', #How long you want the scroll to stay in his case 2m
  size = 10, #How many records you need in each cycle
  body = myquey)

In [51]:
res

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFlBKTDhDQWVhU2JlRDB0RENGeXpDcFEAAAAAAAATxxZBNUZBTDVQY1NjV3pXTTlVbUdZY0NR',
 'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5332, 'relation': 'eq'},
  'max_score': 5.462355,
  'hits': [{'_index': 'elkdemo',
    '_type': '_doc',
    '_id': 's143',
    '_score': 5.462355,
    '_source': {'title': 'Freedom Writers',
     'director': 'Richard LaGravenese',
     'description': 'While her at-risk students are reading classics such as "The Diary of Anne Frank," a teacher asks them to keep journals about their troubled lives.',
     'duration': '124 min',
     'cast': 'Hilary Swank, Patrick Dempsey, Scott Glenn, Imelda Staunton, April L. Hernandez, Mario, Kristin Herrera, Jaclyn Ngan, Sergio Montalvo, Jason Finn, Deance Wyatt, Vanetta Smith'}},
   {'_index': 'elkdemo',
    '_type': '_doc',
    '_id': 's173',
    '_score': 5.462355,
    '_source': {'

In [54]:
page = es.scroll(scroll_id = "FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFlBKTDhDQWVhU2JlRDB0RENGeXpDcFEAAAAAAAATxxZBNUZBTDVQY1NjV3pXTTlVbUdZY0NR", scroll = '10m')
page

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFlBKTDhDQWVhU2JlRDB0RENGeXpDcFEAAAAAAAATxxZBNUZBTDVQY1NjV3pXTTlVbUdZY0NR',
 'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5332, 'relation': 'eq'},
  'max_score': 5.462355,
  'hits': [{'_index': 'elkdemo',
    '_type': '_doc',
    '_id': 's6781',
    '_score': 5.462355,
    '_source': {'title': 'For the Birds',
     'director': 'Richard Miron',
     'description': 'A woman with a passion for fowl keeps a menagerie of birds that draws the ire of neighbors, animal rights advocates and even her own family.',
     'duration': '93 min',
     'cast': 'Kathy Murphy'}},
   {'_index': 'elkdemo',
    '_type': '_doc',
    '_id': 's7691',
    '_score': 5.462355,
    '_source': {'title': 'P.S. I Love You',
     'director': 'Richard LaGravenese',
     'description': 'When she loses her beloved husband to a brain tumor, grieving widow Holly Kennedy learns

In [32]:
counter = 0 
sid =  res["_scroll_id"]
scroll_size = res['hits']['total']
scroll_size = scroll_size['value']
print()


# Start scrolling
while (scroll_size > 0):

    #print("Scrolling...")
    page = es.scroll(scroll_id = sid, scroll = '10m')
    
    #print("Hits : ",len(page["hits"]["hits"]))
    
    # Update the scroll ID
    sid = page['_scroll_id']

    # Get the number of results that we returned in the last scroll
    scroll_size = len(page['hits']['hits'])

    #print("Scroll Size {} ".format(scroll_size))
    
    # Do something with the obtained page
    counter = counter + 1

print("Total Pages  : {}".format(counter))

Total Pages  : 534


## Method 2
- the idea Goes Like this we need to map page number and we divide the search into parts
- say the size was 500
- Page 1 -> 0-10
- Page 2 -> 10-20
- All this time the size is same the query is same all that is changing is from and to word
- think this way you will only query once and create a hashmap with key as page number and value would be sliced Records

In [38]:
res = es.search(
  index = 'elkdemo', #name of the index name
  size = 100, #How many records you need in each cycle
  body = myquey)

In [39]:
data = res["hits"]["hits"]

In [40]:
hashmap = {}

In [41]:
step = 2
hashmap = {}
for i in range(len(data)):
    if i==0:
        hashmap[i] = data[0:step]
    else:
        startIndex = step * i
        EndIndex =  ((i+1) * (step))
        sample = data[startIndex:EndIndex]
        hashmap[i] = sample

# Inserting latitude and Longitude in your Elastic Search as geo_point -> Strinng Mapping to geo_point

In [55]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(e))

Loaded  .. . . . . . . .


In [56]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [69]:
data = {
  'name': 'bellingham, washington, united states',
  'locality': 'bellingham',
  'region': 'washington',
  'subregion': 'whatcom county',
  'country': 'united states',
  'continent': 'north america',
  'type': 'locality',
  'geo': 
    {'lat':48.75, #it should always be lat and lon
     'lon':-122.48
       },
  'postal_code': '98225',
  'zip_plus_4': None,
  'street_address': None,
  'address_line_2': None,
  'most_recent': False,
  'is_primary': True,
  'last_updated': None,
  'last_updated2': None
}

### Creating a Custom Mapping 
Whatever or However the complex data you have while creating index we dont need to provide all feilds mappings. By default elastic Search will assign the feilds their relevant mappings (datatypes). We just have to specify whatever feild you want for example in this example i just wanted to change geo i just specified that feild as geo_point

In [70]:
Sett = { 
    "settings":{ 
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{ 
        "properties":{
            "geo":{ 
                "type":"geo_point"
            }
        }
    }
}

In [71]:
es.indices.create(index='geo_search', ignore=400, body=Sett)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [geo_search/XEuko5O_SvCdCRHOrqMBBw] already exists',
    'index_uuid': 'XEuko5O_SvCdCRHOrqMBBw',
    'index': 'geo_search'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [geo_search/XEuko5O_SvCdCRHOrqMBBw] already exists',
  'index_uuid': 'XEuko5O_SvCdCRHOrqMBBw',
  'index': 'geo_search'},
 'status': 400}

In [72]:
res1 = es.index(index='geo_search', body=data, id=1)
res1

{'_index': 'geo_search',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [ ]:
#Run below query in kibana
GET geo_search/_search
{
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_distance": {
          "distance": "5000km",
          "geo": {
            "lat": 40,
            "lon": -70
          }
        }
      }
    }
  }
}

https://github.com/LisaHJung/Part-2-Understanding-the-relevance-of-your-search-with-Elasticsearch-and-Kibana- for aggregation and query

https://github.com/LisaHJung/Part-3-Running-full-text-queries-and-combined-queries-with-Elasticsearch-and-Kibana for match, match_phrase, multi_match with per-field boosting(search in multiple fields- the terms need not to be in order), boolean query

https://github.com/LisaHJung/Part-4-Running-Aggregations-with-Elasticsearch-and-Kibana for bucket aggregations like date_histogram (with fixed_interval or calender_interval), histogram, range and terms. Metric aggregrations like sum, min, avg, stats etc. And Combined Aggregations using metric aggregations and bucket aggregation both.

https://github.com/LisaHJung/Part-5-Understanding-Mapping-with-Elasticsearch-and-Kibana for mapping and run-time fields